In [2]:
import numpy as np
import matplotlib.pyplot as plt

# 初始化参数
N = 500  # 图像尺寸
Pd = 0.001  # 孔隙核生成概率

# 定义孔隙率列表和每张孔隙率生成的图像数量
porosities = [0.4]
solidVFrac = [1 - porosity for porosity in porosities]
images_per_porosity = 1

# 外层循环，遍历不同的孔隙率（固体率）
for Svf in solidVFrac:
    for _ in range(images_per_porosity):
        # 创建一个全零的二维数组作为起始点
        pore_matrix = np.zeros((N, N), dtype=int)

        # 生成随机数矩阵，用于确定初始孔隙核的位置
        initial_seeds = np.random.rand(N, N) < Pd
        pore_matrix[initial_seeds == True] = 1

        # 定义生长方向
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0), 
                      (1, 1), (1, -1), (-1, 1), (-1, -1)]

        # 定义函数
        def is_valid(x, y):
            return 0 <= x < N and 0 <= y < N

        def grow_pores(pore_matrix):
            new_matrix = pore_matrix.copy()
            growth_probabilities = np.random.rand(*pore_matrix.shape)
            growth_probabilities = growth_probabilities < Svf  # 将随机数转换为生长概率
            for x in range(N):
                for y in range(N):
                    if pore_matrix[x, y] == 1 and growth_probabilities[x, y] == True:
                        for dx, dy in directions:
                            new_x, new_y = x + dx, y + dy
                            if is_valid(new_x, new_y) and new_matrix[new_x, new_y] == 0:
                                new_matrix[new_x, new_y] = 1
            return new_matrix

        # 生长循环
        current_porosity = np.sum(pore_matrix) / (N * N)
        while current_porosity < Svf:
            pore_matrix = grow_pores(pore_matrix)
            current_porosity = np.sum(pore_matrix) / (N * N)

        # 创建一个新的图和子图，用于保存图像
        fig, ax = plt.subplots()

        # 显示图像
        im = ax.imshow(pore_matrix, cmap='gray')

        # 调整子图参数，使得边界紧凑，不包含标题和颜色条
        ax.set_title("")  # 清空标题
        ax.set_xticks([])  # 隐藏x轴的刻度
        ax.set_yticks([])  # 隐藏y轴的刻度
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # 调整子图边界

        # 保存图像，不包含任何额外的元素
        filename = f'Pe{1-Svf}_{_}.png'
        fig.savefig(filename, bbox_inches='tight', pad_inches=0)

        # 关闭图形，释放资源
        plt.close(fig)

# 注意：这段代码没有包含显示图像的部分，因为每次循环都会生成和保存图像，
# 然后关闭图形。如果需要查看图像，可以在循环外部添加显示图像的代码。